In [1]:
import pandas as pd

In [10]:
fname = '../logs/for-bcancer.out-curiosity-with_prefix_perm_map-max_num_nodes=100000-c=0.0010000-v=1-f=1000-opt.txt'
with open(fname, 'r') as f:
    line = f.readline()
    opt = map(lambda x: x.split('~'), line.split(';'))
    print opt
    #opt = map(lambda x: (x[0], int(x[1])), opt)
    #opt = line.split(';')
    print opt

[['{a7<5:0,a2<5:0}', '1'], ['{a1>7:1,a6>7:1}', '1'], ['{a8>7:1,a3<5:0}', '1'], ['{a4>7:1,a6.5-7:0}', '1'], ['{a1<5:1,a6<5:1}', '0'], ['{a6.5-7:1,a3<5:0}', '1'], ['{a6>7:1,a7<5:0}', '1'], ['{a3<5:1,a4<5:0}', '0'], ['{a2.5-7:1,a5.5-7:0}', '1'], ['{a8>7:1,a4.5-7:0}', '1'], ['{a7.5-7:1,a1>7:0}', '0'], ['{a6<5:1,a7<5:0}', '1'], ['{a9<5:0,a7>7:0}', '1'], ['{a6<5:1,a1>7:1}', '0'], ['{a2<5:1,a5.5-7:1}', '1'], ['{a3<5:0,a6<5:1}', '0'], ['{a1>7:0,a8.5-7:0}', '0'], ['default', '1']]
[['{a7<5:0,a2<5:0}', '1'], ['{a1>7:1,a6>7:1}', '1'], ['{a8>7:1,a3<5:0}', '1'], ['{a4>7:1,a6.5-7:0}', '1'], ['{a1<5:1,a6<5:1}', '0'], ['{a6.5-7:1,a3<5:0}', '1'], ['{a6>7:1,a7<5:0}', '1'], ['{a3<5:1,a4<5:0}', '0'], ['{a2.5-7:1,a5.5-7:0}', '1'], ['{a8>7:1,a4.5-7:0}', '1'], ['{a7.5-7:1,a1>7:0}', '0'], ['{a6<5:1,a7<5:0}', '1'], ['{a9<5:0,a7>7:0}', '1'], ['{a6<5:1,a1>7:1}', '0'], ['{a2<5:1,a5.5-7:1}', '1'], ['{a3<5:0,a6<5:1}', '0'], ['{a1>7:0,a8.5-7:0}', '0'], ['default', '1']]


In [11]:
nrules = 0
with open('../data/bcancer.out') as f:
    line = f.readline()
    nrules = len(line.split()) - 1
out = pd.read_csv('../data/bcancer.out', sep=' ', names=['Rule'] + range(nrules))
label = pd.read_csv('../data/bcancer.label', sep=' ', names=['Rule'] + range(nrules))
out.set_index('Rule', inplace=True)
label.set_index('Rule', inplace=True)
out

,0,1,2,3,4,5,6,7,8,9,...,673,674,675,676,677,678,679,680,681,682
Rule,,,,,,,,,,,,,,,,,,,,,
"{a2>7:1,a4.5-7:0}",0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
"{a3<5:0,a4.5-7:0}",0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
"{a5.5-7:1,a3<5:0}",0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
"{a6<5:1,a3<5:1}",1,0,1,0,1,0,0,1,1,1,...,1,1,0,1,1,1,1,0,0,0
"{a6<5:1,a7<5:0}",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
"{a4>7:1,a3>7:1}",0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"{a9.5-7:1,a6.5-7:0}",0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
"{a8>7:1,a4.5-7:0}",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
"{a8<5:1,a6.5-7:0}",1,1,1,0,1,0,1,1,1,1,...,1,1,0,1,1,1,1,0,0,0


In [12]:
captured = set()
preds = []
for (rule, pred) in opt[:-1]:
    cappd = out.ix[rule] == 1
    unfiltered_cappd = out.T[cappd].index
    filtered_cappd = unfiltered_cappd.difference(captured)
    preds += map(lambda x: (x, pred), filtered_cappd)
    captured = captured.union(filtered_cappd)
    
# Handle default rule
_, def_pred = opt[-1]
preds += [(x, def_pred) for x in set(range(nrules)).difference(captured)]

corr = 0
for (ind, pred) in preds:
    corr += label.iloc[pred, ind]
print corr / float(nrules)

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types